In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

PATH = 'beomi.pth'

# 미리 학습한 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
model = torch.load(PATH, map_location=torch.device('cpu'))  # Load on CPU

# 감성 라벨에 대응하는 한글 감정
emotion_labels = ['happiness', 'neutral', 'sadness', 'angry', 'surprise', 'disgust', 'fear']
emotion_korean = {
    'happiness': '행복',
    'neutral': '중립',
    'sadness': '슬픔',
    'angry': '분노',
    'surprise': '놀람',
    'disgust': '혐오',
    'fear': '불안'
}

# 문장을 입력받아 감성을 예측하고 한글 감정을 출력하는 함수
def predict_emotion(sentence):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Move model to correct device

    # 문장을 토큰화하고 모델 입력 형식에 맞게 변환
    inputs = tokenizer(sentence, return_tensors='pt')
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # 모델을 통해 예측 수행
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1).item()

    # 예측된 라벨을 한글 감정으로 변환하여 반환
    predicted_emotion = emotion_labels[predictions]
    return emotion_korean[predicted_emotion]

# 예시 문장을 이용한 감성 예측 및 출력
sample_sentence = input()
predicted_emotion = predict_emotion(sample_sentence)
print(f"입력 문장: {sample_sentence}")
print(f"예측 감정: {predicted_emotion}")


입력 문장: 안녕하세요
예측 감정: 놀람
